In [4]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

# gensim
from gensim import corpora, models, similarities, matutils
import gensim
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.corpora import Dictionary
from nltk.corpus import stopwords
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer

import pickle
import pandas as pd

In [5]:
df = pd.read_pickle("./preprocessed.pkl")

In [6]:
df.head()

,date,tokenized
0,2020-02-28,recent rainy night manhattan kelsea ballerini ...
1,2020-02-22,forget minute two men thought sang milli vanil...
2,2020-02-19,editors note ongoing series explore best buzzw...
3,2020-01-24,continues grow change decided time annual powe...
4,2019-12-12,desiree perez coo roc nation typical day desir...


### code from https://shichaoji.com/tag/topic-modeling-python-lda-visualization-gensim-pyldavis-nltk/

In [7]:
text = df['tokenized']
text_list = [i.split() for i in text]

In [8]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
dictionary = corpora.Dictionary(text_list)
dictionary.save('dictionary.dict')
print(dictionary)

2020-05-18 18:32:45,795 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-05-18 18:32:47,601 : INFO : built Dictionary(36195 unique tokens: ['able', 'absorb', 'acceptable', 'active', 'activity']...) from 2244 documents (total 758088 corpus positions)
2020-05-18 18:32:47,607 : INFO : saving Dictionary object under dictionary.dict, separately None
2020-05-18 18:32:47,962 : INFO : saved dictionary.dict


Dictionary(36195 unique tokens: ['able', 'absorb', 'acceptable', 'active', 'activity']...)


In [9]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_list]
corpora.MmCorpus.serialize('corpus.mm', doc_term_matrix)

2020-05-18 18:32:48,897 : INFO : storing corpus in Matrix Market format to corpus.mm
2020-05-18 18:32:48,931 : INFO : saving sparse matrix to corpus.mm
2020-05-18 18:32:48,932 : INFO : PROGRESS: saving document #0
2020-05-18 18:32:49,533 : INFO : PROGRESS: saving document #1000
2020-05-18 18:32:49,969 : INFO : PROGRESS: saving document #2000
2020-05-18 18:32:50,084 : INFO : saved 2244x36195 matrix, density=0.607% (492687/81221580)
2020-05-18 18:32:50,107 : INFO : saving MmCorpus index to corpus.mm.index


In [10]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Training LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=15, id2word = dictionary, passes=50)

2020-05-18 18:32:50,134 : INFO : using symmetric alpha at 0.06666666666666667
2020-05-18 18:32:50,140 : INFO : using symmetric eta at 0.06666666666666667
2020-05-18 18:32:50,189 : INFO : using serial LDA version on this node
2020-05-18 18:32:50,960 : INFO : running online (multi-pass) LDA training, 15 topics, 50 passes over the supplied corpus of 2244 documents, updating model once every 2000 documents, evaluating perplexity every 2244 documents, iterating 50x with a convergence threshold of 0.001000
2020-05-18 18:32:50,964 : INFO : PROGRESS: pass 0, at document #2000/2244
2020-05-18 18:32:57,432 : INFO : merging changes from 2000 documents into a model of 2244 documents
2020-05-18 18:32:57,498 : INFO : topic #9 (0.067): 0.005*"sony" + 0.004*"artist" + 0.004*"first" + 0.004*"group" + 0.003*"industry" + 0.003*"global" + 0.003*"spotify" + 0.003*"based" + 0.003*"label" + 0.003*"youtube"
2020-05-18 18:32:57,500 : INFO : topic #14 (0.067): 0.006*"artist" + 0.004*"industry" + 0.003*"time" + 

2020-05-18 18:33:15,220 : INFO : topic #0 (0.067): 0.008*"mlc" + 0.007*"live" + 0.006*"aeg" + 0.006*"nation" + 0.006*"awal" + 0.005*"digital" + 0.005*"songwriters" + 0.005*"royalties" + 0.005*"global" + 0.005*"ceo"
2020-05-18 18:33:15,223 : INFO : topic #5 (0.067): 0.011*"swift" + 0.010*"spotify" + 0.008*"taylor" + 0.007*"big" + 0.006*"last" + 0.006*"braun" + 0.006*"eone" + 0.005*"machine" + 0.005*"songs" + 0.005*"statement"
2020-05-18 18:33:15,227 : INFO : topic #13 (0.067): 0.008*"digital" + 0.007*"content" + 0.007*"global" + 0.006*"marketing" + 0.006*"media" + 0.005*"group" + 0.005*"based" + 0.004*"platform" + 0.004*"tencent" + 0.004*"strategy"
2020-05-18 18:33:15,230 : INFO : topic diff=0.680377, rho=0.492545
2020-05-18 18:33:15,235 : INFO : PROGRESS: pass 3, at document #2000/2244
2020-05-18 18:33:20,253 : INFO : merging changes from 2000 documents into a model of 2244 documents
2020-05-18 18:33:20,293 : INFO : topic #9 (0.067): 0.018*"sony" + 0.006*"group" + 0.006*"first" + 0.005

2020-05-18 18:33:34,221 : INFO : topic #4 (0.067): 0.025*"album" + 0.014*"songs" + 0.013*"platinum" + 0.012*"top" + 0.010*"albums" + 0.009*"chart" + 0.007*"released" + 0.007*"catalog" + 0.007*"certified" + 0.007*"hipgnosis"
2020-05-18 18:33:34,224 : INFO : topic #5 (0.067): 0.013*"swift" + 0.010*"spotify" + 0.009*"taylor" + 0.007*"big" + 0.007*"last" + 0.006*"rights" + 0.006*"braun" + 0.006*"eone" + 0.006*"would" + 0.005*"statement"
2020-05-18 18:33:34,226 : INFO : topic #3 (0.067): 0.010*"industry" + 0.009*"digital" + 0.008*"creators" + 0.007*"services" + 0.007*"kobalt" + 0.006*"revenue" + 0.006*"management" + 0.006*"triller" + 0.005*"chief" + 0.005*"group"
2020-05-18 18:33:34,229 : INFO : topic #12 (0.067): 0.012*"concord" + 0.008*"group" + 0.006*"ebay" + 0.006*"jazz" + 0.005*"cox" + 0.005*"future" + 0.005*"richter" + 0.005*"london" + 0.004*"max" + 0.004*"film"
2020-05-18 18:33:34,232 : INFO : topic diff=0.188542, rho=0.374713
2020-05-18 18:33:34,237 : INFO : PROGRESS: pass 6, at doc

2020-05-18 18:33:52,340 : INFO : topic #3 (0.067): 0.011*"industry" + 0.009*"digital" + 0.009*"creators" + 0.007*"services" + 0.006*"kobalt" + 0.006*"management" + 0.006*"revenue" + 0.005*"chief" + 0.005*"technology" + 0.005*"triller"
2020-05-18 18:33:52,343 : INFO : topic #8 (0.067): 0.015*"songs" + 0.011*"hipgnosis" + 0.008*"fund" + 0.006*"wave" + 0.006*"primary" + 0.006*"million" + 0.006*"nashville" + 0.006*"michael" + 0.006*"share" + 0.006*"peermusic"
2020-05-18 18:33:52,345 : INFO : topic #6 (0.067): 0.020*"spotify" + 0.010*"streaming" + 0.006*"labels" + 0.005*"service" + 0.005*"content" + 0.005*"apple" + 0.005*"market" + 0.004*"platform" + 0.004*"users" + 0.004*"services"
2020-05-18 18:33:52,348 : INFO : topic #14 (0.067): 0.014*"film" + 0.013*"umg" + 0.011*"piracy" + 0.010*"studiocanal" + 0.010*"rights" + 0.009*"creators" + 0.007*"nyege" + 0.006*"tap" + 0.006*"spinal" + 0.006*"complaint"
2020-05-18 18:33:52,351 : INFO : topic #12 (0.067): 0.012*"concord" + 0.008*"group" + 0.007*

2020-05-18 18:34:10,727 : INFO : PROGRESS: pass 11, at document #2244/2244
2020-05-18 18:34:11,721 : INFO : merging changes from 244 documents into a model of 2244 documents
2020-05-18 18:34:11,777 : INFO : topic #4 (0.067): 0.028*"album" + 0.015*"songs" + 0.013*"top" + 0.011*"platinum" + 0.011*"chart" + 0.010*"albums" + 0.009*"released" + 0.007*"single" + 0.006*"catalog" + 0.006*"artist"
2020-05-18 18:34:11,779 : INFO : topic #2 (0.067): 0.023*"africa" + 0.017*"boomplay" + 0.012*"african" + 0.009*"partnership" + 0.009*"sony" + 0.008*"nigeria" + 0.008*"mckay" + 0.008*"davidson" + 0.008*"mayer" + 0.007*"users"
2020-05-18 18:34:11,781 : INFO : topic #3 (0.067): 0.012*"industry" + 0.009*"creators" + 0.009*"digital" + 0.007*"services" + 0.006*"kobalt" + 0.006*"management" + 0.006*"support" + 0.005*"technology" + 0.005*"chief" + 0.005*"soundcloud"
2020-05-18 18:34:11,784 : INFO : topic #13 (0.067): 0.011*"digital" + 0.009*"content" + 0.009*"media" + 0.008*"marketing" + 0.007*"global" + 0.00

...

2020-05-18 18:36:46,337 : INFO : topic #4 (0.067): 0.028*"album" + 0.014*"songs" + 0.012*"top" + 0.010*"chart" + 0.010*"platinum" + 0.010*"released" + 0.009*"albums" + 0.008*"artist" + 0.008*"hit" + 0.008*"single"
2020-05-18 18:36:46,339 : INFO : topic #13 (0.067): 0.011*"digital" + 0.010*"content" + 0.010*"media" + 0.009*"marketing" + 0.008*"global" + 0.007*"tiktok" + 0.005*"strategy" + 0.005*"platform" + 0.005*"social" + 0.005*"entertainment"
2020-05-18 18:36:46,341 : INFO : topic diff=0.032682, rho=0.159878
2020-05-18 18:36:46,346 : INFO : PROGRESS: pass 38, at document #2000/2244
2020-05-18 18:36:49,869 : INFO : merging changes from 2000 documents into a model of 2244 documents
2020-05-18 18:36:49,911 : INFO : topic #5 (0.067): 0.013*"swift" + 0.009*"taylor" + 0.008*"spotify" + 0.008*"rights" + 0.007*"last" + 0.007*"big" + 0.007*"would" + 0.006*"statement" + 0.005*"deal" + 0.005*"braun"
2020-05-18 18:36:49,914 : INFO : topic #12 (0.067): 0.014*"concord" + 0.007*"group" + 0.007*"jaz

2020-05-18 18:37:01,228 : INFO : topic #6 (0.067): 0.021*"spotify" + 0.011*"streaming" + 0.006*"service" + 0.006*"labels" + 0.005*"apple" + 0.005*"content" + 0.005*"market" + 0.005*"platform" + 0.005*"users" + 0.005*"million"
2020-05-18 18:37:01,231 : INFO : topic #12 (0.067): 0.013*"concord" + 0.007*"group" + 0.007*"jazz" + 0.006*"ebay" + 0.006*"viagogo" + 0.005*"cox" + 0.005*"future" + 0.005*"london" + 0.004*"richter" + 0.004*"max"
2020-05-18 18:37:01,233 : INFO : topic #2 (0.067): 0.023*"africa" + 0.017*"boomplay" + 0.012*"african" + 0.009*"partnership" + 0.008*"nigeria" + 0.008*"sony" + 0.008*"mayer" + 0.008*"davidson" + 0.008*"mckay" + 0.006*"users"
2020-05-18 18:37:01,236 : INFO : topic diff=0.031451, rho=0.154080
2020-05-18 18:37:01,240 : INFO : PROGRESS: pass 41, at document #2000/2244
2020-05-18 18:37:04,728 : INFO : merging changes from 2000 documents into a model of 2244 documents
2020-05-18 18:37:04,779 : INFO : topic #3 (0.067): 0.013*"industry" + 0.010*"creators" + 0.009*

2020-05-18 18:37:17,862 : INFO : topic #13 (0.067): 0.011*"digital" + 0.010*"content" + 0.010*"media" + 0.009*"marketing" + 0.008*"global" + 0.007*"tiktok" + 0.005*"strategy" + 0.005*"social" + 0.005*"platform" + 0.005*"entertainment"
2020-05-18 18:37:17,864 : INFO : topic #0 (0.067): 0.013*"live" + 0.009*"nation" + 0.009*"songwriters" + 0.007*"tickets" + 0.006*"publishers" + 0.006*"ticketing" + 0.006*"sound" + 0.006*"aeg" + 0.006*"festival" + 0.005*"ticket"
2020-05-18 18:37:17,867 : INFO : topic #11 (0.067): 0.010*"team" + 0.008*"records" + 0.008*"label" + 0.007*"group" + 0.007*"artist" + 0.007*"president" + 0.006*"global" + 0.006*"creative" + 0.006*"publishing" + 0.006*"warner"
2020-05-18 18:37:17,869 : INFO : topic #5 (0.067): 0.013*"swift" + 0.009*"taylor" + 0.009*"spotify" + 0.008*"rights" + 0.007*"last" + 0.007*"big" + 0.006*"would" + 0.006*"statement" + 0.006*"braun" + 0.005*"deal"
2020-05-18 18:37:17,872 : INFO : topic diff=0.030242, rho=0.148870
2020-05-18 18:37:17,876 : INFO 

2020-05-18 18:37:34,756 : INFO : merging changes from 244 documents into a model of 2244 documents
2020-05-18 18:37:34,796 : INFO : topic #8 (0.067): 0.017*"songs" + 0.015*"hipgnosis" + 0.009*"fund" + 0.007*"law" + 0.007*"school" + 0.006*"wave" + 0.006*"share" + 0.006*"primary" + 0.006*"peermusic" + 0.006*"million"
2020-05-18 18:37:34,798 : INFO : topic #0 (0.067): 0.013*"live" + 0.009*"nation" + 0.009*"songwriters" + 0.007*"tickets" + 0.006*"publishers" + 0.006*"ticketing" + 0.006*"sound" + 0.006*"aeg" + 0.006*"festival" + 0.005*"ticket"
2020-05-18 18:37:34,801 : INFO : topic #9 (0.067): 0.021*"sony" + 0.013*"umg" + 0.011*"group" + 0.011*"universal" + 0.007*"publishing" + 0.007*"spirit" + 0.006*"first" + 0.005*"revenues" + 0.005*"vivendi" + 0.005*"songs"
2020-05-18 18:37:34,803 : INFO : topic #7 (0.067): 0.007*"centtrip" + 0.007*"election" + 0.007*"majority" + 0.006*"win" + 0.005*"markets" + 0.005*"brexit" + 0.005*"likely" + 0.005*"eventim" + 0.005*"sterling" + 0.004*"asleep"
2020-05-

2020-05-18 18:37:50,984 : INFO : topic diff=0.028343, rho=0.139861
2020-05-18 18:37:52,337 : INFO : -8.301 per-word bound, 315.5 perplexity estimate based on a held-out corpus of 244 documents with 77060 words
2020-05-18 18:37:52,339 : INFO : PROGRESS: pass 49, at document #2244/2244
2020-05-18 18:37:52,904 : INFO : merging changes from 244 documents into a model of 2244 documents
2020-05-18 18:37:52,955 : INFO : topic #3 (0.067): 0.013*"industry" + 0.009*"creators" + 0.009*"digital" + 0.008*"services" + 0.008*"soundcloud" + 0.007*"support" + 0.006*"management" + 0.006*"kobalt" + 0.005*"technology" + 0.005*"independent"
2020-05-18 18:37:52,957 : INFO : topic #11 (0.067): 0.010*"team" + 0.008*"records" + 0.008*"label" + 0.007*"group" + 0.007*"artist" + 0.007*"president" + 0.006*"global" + 0.006*"creative" + 0.006*"publishing" + 0.006*"warner"
2020-05-18 18:37:52,959 : INFO : topic #10 (0.067): 0.008*"time" + 0.007*"going" + 0.006*"artist" + 0.006*"get" + 0.005*"would" + 0.005*"work" + 0

In [11]:
print(ldamodel.print_topics(num_topics=15, num_words=10))

2020-05-18 18:37:53,014 : INFO : topic #0 (0.067): 0.013*"live" + 0.009*"songwriters" + 0.009*"nation" + 0.007*"tickets" + 0.006*"publishers" + 0.006*"ticketing" + 0.006*"sound" + 0.006*"aeg" + 0.006*"festival" + 0.005*"ticket"
2020-05-18 18:37:53,017 : INFO : topic #1 (0.067): 0.024*"radio" + 0.012*"bbc" + 0.008*"stations" + 0.007*"tunein" + 0.006*"appeal" + 0.006*"warner" + 0.006*"court" + 0.006*"gmr" + 0.005*"london" + 0.005*"rmlc"
2020-05-18 18:37:53,021 : INFO : topic #2 (0.067): 0.023*"africa" + 0.017*"boomplay" + 0.013*"african" + 0.009*"partnership" + 0.008*"nigeria" + 0.008*"sony" + 0.008*"mayer" + 0.008*"davidson" + 0.007*"mckay" + 0.006*"users"
2020-05-18 18:37:53,025 : INFO : topic #3 (0.067): 0.013*"industry" + 0.009*"creators" + 0.009*"digital" + 0.008*"services" + 0.008*"soundcloud" + 0.007*"support" + 0.006*"management" + 0.006*"kobalt" + 0.005*"technology" + 0.005*"independent"
2020-05-18 18:37:53,031 : INFO : topic #4 (0.067): 0.028*"album" + 0.014*"songs" + 0.012*"to

[(0, '0.013*"live" + 0.009*"songwriters" + 0.009*"nation" + 0.007*"tickets" + 0.006*"publishers" + 0.006*"ticketing" + 0.006*"sound" + 0.006*"aeg" + 0.006*"festival" + 0.005*"ticket"'), (1, '0.024*"radio" + 0.012*"bbc" + 0.008*"stations" + 0.007*"tunein" + 0.006*"appeal" + 0.006*"warner" + 0.006*"court" + 0.006*"gmr" + 0.005*"london" + 0.005*"rmlc"'), (2, '0.023*"africa" + 0.017*"boomplay" + 0.013*"african" + 0.009*"partnership" + 0.008*"nigeria" + 0.008*"sony" + 0.008*"mayer" + 0.008*"davidson" + 0.007*"mckay" + 0.006*"users"'), (3, '0.013*"industry" + 0.009*"creators" + 0.009*"digital" + 0.008*"services" + 0.008*"soundcloud" + 0.007*"support" + 0.006*"management" + 0.006*"kobalt" + 0.005*"technology" + 0.005*"independent"'), (4, '0.028*"album" + 0.014*"songs" + 0.012*"top" + 0.010*"chart" + 0.010*"released" + 0.010*"platinum" + 0.009*"albums" + 0.008*"artist" + 0.008*"hit" + 0.008*"single"'), (5, '0.013*"swift" + 0.009*"taylor" + 0.009*"spotify" + 0.008*"rights" + 0.007*"last" + 0.00

In [12]:
for i in ldamodel.print_topics(): 
    for j in i: print(j)

2020-05-18 18:37:53,162 : INFO : topic #0 (0.067): 0.013*"live" + 0.009*"songwriters" + 0.009*"nation" + 0.007*"tickets" + 0.006*"publishers" + 0.006*"ticketing" + 0.006*"sound" + 0.006*"aeg" + 0.006*"festival" + 0.005*"ticket"
2020-05-18 18:37:53,166 : INFO : topic #1 (0.067): 0.024*"radio" + 0.012*"bbc" + 0.008*"stations" + 0.007*"tunein" + 0.006*"appeal" + 0.006*"warner" + 0.006*"court" + 0.006*"gmr" + 0.005*"london" + 0.005*"rmlc"
2020-05-18 18:37:53,171 : INFO : topic #2 (0.067): 0.023*"africa" + 0.017*"boomplay" + 0.013*"african" + 0.009*"partnership" + 0.008*"nigeria" + 0.008*"sony" + 0.008*"mayer" + 0.008*"davidson" + 0.007*"mckay" + 0.006*"users"
2020-05-18 18:37:53,173 : INFO : topic #3 (0.067): 0.013*"industry" + 0.009*"creators" + 0.009*"digital" + 0.008*"services" + 0.008*"soundcloud" + 0.007*"support" + 0.006*"management" + 0.006*"kobalt" + 0.005*"technology" + 0.005*"independent"
2020-05-18 18:37:53,178 : INFO : topic #4 (0.067): 0.028*"album" + 0.014*"songs" + 0.012*"to

0
0.013*"live" + 0.009*"songwriters" + 0.009*"nation" + 0.007*"tickets" + 0.006*"publishers" + 0.006*"ticketing" + 0.006*"sound" + 0.006*"aeg" + 0.006*"festival" + 0.005*"ticket"
1
0.024*"radio" + 0.012*"bbc" + 0.008*"stations" + 0.007*"tunein" + 0.006*"appeal" + 0.006*"warner" + 0.006*"court" + 0.006*"gmr" + 0.005*"london" + 0.005*"rmlc"
2
0.023*"africa" + 0.017*"boomplay" + 0.013*"african" + 0.009*"partnership" + 0.008*"nigeria" + 0.008*"sony" + 0.008*"mayer" + 0.008*"davidson" + 0.007*"mckay" + 0.006*"users"
3
0.013*"industry" + 0.009*"creators" + 0.009*"digital" + 0.008*"services" + 0.008*"soundcloud" + 0.007*"support" + 0.006*"management" + 0.006*"kobalt" + 0.005*"technology" + 0.005*"independent"
4
0.028*"album" + 0.014*"songs" + 0.012*"top" + 0.010*"chart" + 0.010*"released" + 0.010*"platinum" + 0.009*"albums" + 0.008*"artist" + 0.008*"hit" + 0.008*"single"
5
0.013*"swift" + 0.009*"taylor" + 0.009*"spotify" + 0.008*"rights" + 0.007*"last" + 0.007*"big" + 0.006*"would" + 0.006*"s

In [13]:
#saving model
ldamodel.save('topic.model')

2020-05-18 18:37:53,238 : INFO : saving LdaState object under topic.model.state, separately None
2020-05-18 18:37:53,338 : INFO : saved topic.model.state
2020-05-18 18:37:53,505 : INFO : saving LdaModel object under topic.model, separately ['expElogbeta', 'sstats']
2020-05-18 18:37:53,507 : INFO : storing np array 'expElogbeta' to topic.model.expElogbeta.npy
2020-05-18 18:37:53,570 : INFO : not storing attribute id2word
2020-05-18 18:37:53,571 : INFO : not storing attribute dispatcher
2020-05-18 18:37:53,573 : INFO : not storing attribute state
2020-05-18 18:37:53,588 : INFO : saved topic.model


### visualization

In [14]:
import pyLDAvis.gensim
import gensim
pyLDAvis.enable_notebook()


In [15]:
d = gensim.corpora.Dictionary.load('dictionary.dict')
c = gensim.corpora.MmCorpus('corpus.mm')
lda = gensim.models.LdaModel.load('topic.model')

2020-05-18 18:37:54,143 : INFO : loading Dictionary object from dictionary.dict
2020-05-18 18:37:54,193 : INFO : loaded dictionary.dict
2020-05-18 18:37:54,200 : INFO : loaded corpus index from corpus.mm.index
2020-05-18 18:37:54,204 : INFO : initializing cython corpus reader from corpus.mm
2020-05-18 18:37:54,218 : INFO : accepted corpus with 2244 documents, 36195 features, 492687 non-zero entries
2020-05-18 18:37:54,220 : INFO : loading LdaModel object from topic.model
2020-05-18 18:37:54,236 : INFO : loading expElogbeta from topic.model.expElogbeta.npy with mmap=None
2020-05-18 18:37:54,247 : INFO : setting ignored attribute id2word to None
2020-05-18 18:37:54,249 : INFO : setting ignored attribute dispatcher to None
2020-05-18 18:37:54,250 : INFO : setting ignored attribute state to None
2020-05-18 18:37:54,252 : INFO : loaded topic.model
2020-05-18 18:37:54,253 : INFO : loading LdaState object from topic.model.state
2020-05-18 18:37:54,299 : INFO : loaded topic.model.state


In [16]:
#need to remove additional stopwords
data = pyLDAvis.gensim.prepare(lda, c, d)
data

2020-05-18 18:37:59,492 : INFO : NumExpr defaulting to 4 threads.
//anaconda3/envs/metis/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.142898 -0.107463       1        1  24.641861
11    -0.141713  0.009627       2        1  20.323399
6     -0.145386  0.029240       3        1  19.271044
13    -0.128047  0.150931       4        1   8.201403
0     -0.030697  0.173813       5        1   5.436639
3     -0.100898  0.146406       6        1   4.895654
4     -0.081048 -0.244616       7        1   4.803760
9     -0.095167 -0.058238       8        1   4.453715
5     -0.028977 -0.046662       9        1   2.616150
8      0.041140 -0.104808      10        1   1.742708
12     0.062029  0.018400      11        1   1.430079
1      0.139938 -0.003205      12        1   1.117687
2      0.151801  0.025153      13        1   0.424179
14     0.239289  0.009536      14        1   0.412631
7      0.260632  0.001887      15        1   0.229086, topic_info=     Category         Freq       Term        Total  loglift  logprob
3849  Default  3493.000000    spotify  3493.000000  30.0000  30.0000
11    Default  1623.000000      album  1623.000000  29.0000  29.0000
356   Default  1046.000000      radio  1046.000000  28.0000  28.0000
3825  Default  1622.000000       sony  1622.000000  27.0000  27.0000
425   Default  1577.000000      songs  1577.000000  26.0000  26.0000
...       ...          ...        ...          ...      ...      ...
2391  Topic15     4.813082      event   324.931549   1.8666  -5.8884
664   Topic15     4.811230        end   794.177734   0.9725  -5.8888
764   Topic15     4.958493   industry  2439.896973  -0.1198  -5.8586
532   Topic15     4.786644  according   857.552307   0.8906  -5.8939
233   Topic15     4.475138       lead   393.235962   1.6030  -5.9612

[1089 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
12735     10  0.972963      aaliyah
1556      11  0.102685        abbey
1556      12  0.872819        abbey
0          1  0.451670         able
0          2  0.073497         able
...      ...       ...          ...
23569     13  0.849140       zambia
4237       7  0.919527         zane
31467      4  0.982335  zebralution
23331      8  0.971105      zelnick
23093      4  0.971048        zhang

[3338 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 12, 7, 14, 1, 4, 5, 10, 6, 9, 13, 2, 3, 15, 8])